In [1]:
import os
os.environ["OPENAI_API_KEY"] = "NA" # This satisfies the validator
os.environ["OTEL_SDK_DISABLED"] = "true"


# install the necessary libraries 
# !pip install crewai crewai_tools langchain_community
#pip install nest_asyncio
# Warning control
import warnings
warnings.filterwarnings('ignore')

import nest_asyncio
nest_asyncio.apply()


In [2]:
llm = "ollama/llama3.2"
llm = "ollama/mistral"


In [3]:

from crewai import Agent, Task, Crew, LLM
from crewai.flow.flow import Flow, listen, start
from crewai_tools import DirectoryReadTool, FileReadTool
import re
import os

class DynamicKnowledgeFlow(Flow):
    @start()
    def list_files(self):
        print("--- Step 1: Scanning Directory ---")
        dir_tool = DirectoryReadTool(directory='./knowledge')
        files_string = dir_tool.run()
        
        # IMPROVED PARSING LOGIC:
        # DirectoryReadTool returns formatted string like:
        # "File paths: \n-./knowledge/file1.txt\n-./knowledge/file2.txt"
        # Note: File paths are prefixed with "-" (dash)
        lines = [f.strip() for f in files_string.split('\n') if f.strip()]
        
        # Filter out header lines and extract actual file paths
        file_paths = []
        for line in lines:
            # Skip header lines like "File paths:", "Files:", etc.
            if line.lower().startswith('file') and ':' in line:
                continue
            
            # Remove dash prefix if present (DirectoryReadTool uses "-./path" format)
            path = line.strip()
            if path.startswith('-'):
                path = path[1:].strip()
            
            # Remove any leading/trailing quotes
            path = path.strip('"\'')
            
            # Check if line looks like a file path
            if path.startswith('./') or path.startswith('/') or '/' in path:
                # Validate path exists or is a valid relative path
                if os.path.exists(path) or path.startswith('./'):
                    file_paths.append(path)
        
        print(f"Found {len(file_paths)} files: {file_paths}")
        return file_paths[0] if file_paths else None

    @listen(list_files)
    def read_files_dynamically(self, first_file_path):
        """
        Step 2: Read file directly (no Crew inside Flow to avoid RecursionError).
        Agent + Task run in the next cell on this content.
        """
        if not first_file_path:
            return "No files found in directory."
        
        if not os.path.exists(first_file_path):
            return f"File not found: {first_file_path}"
            
        print(f"--- Step 2: Reading file: {first_file_path} ---")
        
        try:
            file_tool = FileReadTool(file_path=first_file_path)
            content = file_tool.run()
            print(content)
            # Plain string only (path + content) so Flow does not recurse
            return f"FILE_PATH:{first_file_path}\n\nCONTENT:\n{content}"
        except Exception as e:
            return f"FILE_PATH:{first_file_path}\n\nERROR:{str(e)}"

In [4]:
# Execute the Flow (no Crew inside — avoids RecursionError)
flow = DynamicKnowledgeFlow()
result = flow.kickoff()



Flow started with ID: dacfc0d9-b4c4-479f-b582-12d134c4f685

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: DynamicKnowledgeFlow                                                                                     │
│  ID: dacfc0d9-b4c4-479f-b582-12d134c4f685                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Flow started with ID: dacfc0d9-b4c4-479f-b582-12d134c4f685


--- Step 2: Reading file: ./knowledge/deeplearning_batch.txt ---

Using Tool: Read a file's content

Output()

The Batch | DeepLearning.AI | AI News & Insights ✨ New course! Enroll in Agent Skills with Anthropic Explore 
Courses AI Newsletter The Batch Andrew's Letter Data Points ML Research Blog ✨ AI Dev x SF 26 Community Forum 
Events Ambassadors Ambassador Spotlight Resources Membership Start Learning What Matters in AI Right Now Subscribe 
First Name Last Name Where do you live? Select a country What is your role? Please select Keep me updated on the 
latest news, events, and courses Subscribe 🗞️   Stay updated with weekly AI News and Insights delivered to your 
inbox Weekly Issues Andrew's Letters Data Points ML Research Business Science Culture Hardware AI Careers About Jan
30, 2026 Agents Go Shopping, Intelligence Redefined, Better Text in Pictures, Higher Engagement Means Worse 
Alignment The Batch AI News and Insights: U.S. policies are driving allies away from using American AI technology. 
Read more Popular Articles Introducing DeepLearning.AI Pro Oct 31, 2025 1 min read Reinforcement Learning Heats Up,
White House Orders Muscular AI Policy, Computer Use Gains Momentum, Fine Control of Fine-Tuning Jan 29, 2025 13 min
read DeepSeek Sharpens Its Reasoning: DeepSeek-R1, an affordable rival to OpenAI’s o1 Jan 22, 2025 4 min read Jan 
23, 2026 Self-Driving Reasoning Models, ChatGPT Adds Ads, Apple’s Deal with Google, 3D Generation Pronto The Batch 
AI News and Insights: How can businesses go beyond using AI for incremental efficiency gains to create 
transformative impact? Jan 16, 2026 Governments vs. Grok, Meta Buys Agent Tech, Healthcare Chatbots, Limits of 
AI-Powered Retrieval The Batch AI News and Insights: Many people are fighting the growth of data centers because 
they could increase CO2 emissions, electricity prices, and water use. Jan 09, 2026 LLMs Go To Confession, Automated
Scientific Research, What Copilot Users Want, Reasoning For Less The Batch AI News and Insights: We just launched a
course that shows people who have never coded before, in less than 30 minutes, how to describe an idea for an app 
and build it using AI. Jan 02, 2026 New Year Special! Hopes for 2026 from David Cox, Adji Bousso Dieng, Juan M. 
Lavista Ferres, Tanmay Gupta, Pengtao Xie, Sharon Zhou The Batch AI News and Insights: Happy 2026! Will this be the
year we finally achieve AGI? I’d like to propose a new version of the Turing Test, which I’ll call the Turing-AGI 
Test, to see if we’ve achieved this. Dec 26, 2025 Top Stories of 2025! Big AI Poaches Talent, Reasoning Models 
Boost Performance, Agents Write Code, Data Centers Drive GDP, China Turns the Tables The Batch AI News and 
Insights: Another year of rapid AI advances has created more opportunities than ever for anyone — including those 
just entering the field — to build software. Dec 17, 2025 OpenAI’s Answer to Gemini 3, Runway’s Interactive Worlds,
Disney’s Alliance With OpenAI, Adapting LLMs for Low-Data Domains The Batch AI News and Insights: As amazing as 
LLMs are, improving their knowledge today involves a more piecemeal process than is widely appreciated. Dec 10, 
2025 Claude Opus 4.5 Saves Tokens, White House Boosts AI-Powered Science, Amazon Exposes Nova 2 Pro Checkpoints, 
Small Models Solve Hard Puzzles The Batch AI News and Insights: If you have not yet built an agentic workflow, I 
encourage you to try doing so, using the simple recipe I’ll share here! Dec 03, 2025 Meta’s Open 3D Pipeline, World
Labs’ Virtual Spaces, Baidu’s Multimodal Models, Coordinating Robot Teams The Batch AI News and Insights: Separate 
reports by the publicity firm Edelman and Pew Research show that Americans, and more broadly large parts of Europe 
and the western world, do not trust AI and are not excited about it. Nov 26, 2025 Google Rules Arena Leaderboards, 
Microsoft+Anthropic, Record Labels Back AI Music, Personality Control for LLMs The Batch AI News and Insights: Is 
there an AI bubble? With the massive number of dollars going into AI infrastructure such as OpenAI’s $1.4 trillion 
plan and Nv

--- Step 1: Scanning Directory ---
Using Tool: List files in directory
Found 1 files: ['./knowledge/deeplearning_batch.txt']


╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: DynamicKnowledgeFlow                                                                                     │
│  ID: dacfc0d9-b4c4-479f-b582-12d134c4f685                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯